https://deallen7.medium.com/how-to-apply-a-function-with-two-or-more-arguments-in-a-pandas-dataframe-ea80abe68976

In [57]:
import pandas as pd
import requests
import numpy as np

In [58]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

In [59]:
r = requests.get(url)
json = r.json()

In [60]:
elements_df = pd.DataFrame(json['elements'])

In [61]:
slim_elements_df = elements_df[['web_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

In [62]:
slim_elements_df.head()

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Balogun,1,4,0.7,45,0,6810,0.0,0
1,Cédric,1,2,0.4,40,0,2935,0.0,0
2,M.Elneny,1,3,0.1,44,0,2796,0.0,0
3,Fábio Vieira,1,3,0.1,54,0,587,0.0,0
4,Gabriel,1,2,25.0,49,24,87136,0.4,2


In [63]:
def playerSubbed(minutes):
    max_possible = slim_elements_df.minutes.max()
    
    if max_possible - minutes > 0:
        sub_status = 'subbed'
    else:
        sub_status = 'not_subbed'
    return sub_status

In [64]:
slim_elements_df['sub_status'] = slim_elements_df.minutes.apply(playerSubbed)

C:\Users\mbieg\AppData\Local\Temp\ipykernel_12264\529537631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['sub_status'] = slim_elements_df.minutes.apply(playerSubbed)


In [65]:
slim_elements_df.loc[slim_elements_df.sub_status == 'not_subbed']

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status
5,Havertz,1,3,3.7,74,180,11482,0.5,4,not_subbed
14,Thomas,1,3,0.4,50,180,4600,1.0,5,not_subbed
16,Ramsdale,1,1,17.7,50,180,73980,1.6,8,not_subbed
19,Saliba,1,2,25.2,51,180,557086,2.5,13,not_subbed
26,White,1,2,9.4,55,180,72901,1.5,8,not_subbed
...,...,...,...,...,...,...,...,...,...,...
660,Lemina,20,3,0.0,50,180,926,0.6,3,not_subbed
661,Matheus N.,20,3,0.2,50,180,4234,0.2,1,not_subbed
662,Neto,20,3,0.3,55,180,9329,0.5,3,not_subbed
664,José Sá,20,1,1.1,50,180,9123,0.6,3,not_subbed


In [66]:
slim_elements_df.selected_by_percent = slim_elements_df.selected_by_percent.astype(float)

C:\Users\mbieg\AppData\Local\Temp\ipykernel_12264\2115023063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df.selected_by_percent = slim_elements_df.selected_by_percent.astype(float)


In [67]:
def findValue (selected_by_percent, total_points):
    selected_by_limit = 10
    top_90_percentile = slim_elements_df.total_points.quantile(0.9)
    
    if (selected_by_percent <= selected_by_limit) & (total_points >= top_90_percentile) :
        scout_flag = 'interesting'
    else :
        scout_flag = 'meh'
    return scout_flag

In [68]:
slim_elements_df['scout_flag'] = slim_elements_df.apply(lambda x: findValue(x['selected_by_percent'], x['total_points']),axis=1)

C:\Users\mbieg\AppData\Local\Temp\ipykernel_12264\3130793658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['scout_flag'] = slim_elements_df.apply(lambda x: findValue(x['selected_by_percent'], x['total_points']),axis=1)


In [69]:
display(slim_elements_df)
slim_elements_df.loc[(slim_elements_df.scout_flag=='interesting')&(slim_elements_df.sub_status=='not_subbed')]

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status,scout_flag
0,Balogun,1,4,0.7,45,0,6810,0.0,0,subbed,meh
1,Cédric,1,2,0.4,40,0,2935,0.0,0,subbed,meh
2,M.Elneny,1,3,0.1,44,0,2796,0.0,0,subbed,meh
3,Fábio Vieira,1,3,0.1,54,0,587,0.0,0,subbed,meh
4,Gabriel,1,2,25.0,49,24,87136,0.4,2,subbed,meh
...,...,...,...,...,...,...,...,...,...,...,...
667,N.Semedo,20,2,0.4,45,180,9058,0.4,2,not_subbed,meh
668,Toti,20,2,0.1,45,19,261,0.2,1,subbed,meh
669,Boubacar Traore,20,3,0.5,45,0,4113,0.0,0,subbed,meh
670,Cunha,20,4,0.6,55,166,31114,0.7,4,subbed,meh


,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status,scout_flag
26,White,1,2,9.4,55,180,72901,1.5,8,not_subbed,interesting
44,Digne,2,2,1.9,45,180,69612,2.0,9,not_subbed,interesting
51,Martinez,2,1,8.1,50,180,62402,1.8,9,not_subbed,interesting
83,Neto,3,1,0.9,45,180,16881,2.7,12,not_subbed,interesting
91,Solanke,3,4,2.9,65,180,31566,1.8,12,not_subbed,interesting
108,Collins,4,2,0.3,45,180,6571,1.6,7,not_subbed,interesting
113,Flekken,4,1,2.2,45,180,44008,1.8,8,not_subbed,interesting
115,Henry,4,2,5.5,46,180,307148,2.0,9,not_subbed,interesting
124,Pinnock,4,2,2.1,45,180,23609,1.8,8,not_subbed,interesting
241,Disasi,7,2,5.5,50,180,395762,2.2,11,not_subbed,interesting
